# Download Chesapeake Bay Program Water Quality Dataset

In [1]:
import os
import requests
import tempfile
from datetime import datetime

import pandas as pd
from pathlib import Path


In [2]:
# Get root path of the project
REPO_ROOT = Path(Path.cwd()).parent.parent

start_date = datetime(2003, 1, 1)
end_date = datetime(2022, 12, 31)


Temp, Sal, DO download:

https://datahub.chesapeakebay.net/api.Tab/WaterQuality/WaterQuality/12-8-2018/12-8-2023/0/2,4,6/12,13,15,35,36,2,3,7,33,34,23,24/HUC8/2,4,6,7,8,9,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,60/31,123,83

In [3]:
def format_request_temponly(start, end):
    '''
    Creating URL string for requesting water quality data from the Chesapeake Bay 
    Program.
    
    Start and end date formatted as `month-day-year`, or '%m-%d-%Y' 
    using https://strftime.org/
    '''
    return (
        'https://datahub.chesapeakebay.net/api.CSV/WaterQuality/WaterQuality/'
        f'{start}/{end}/0/2,4,6/12,13,15,35,36,2,3,7,33,34,23,24/HUC8/'
        '2,4,6,7,8,9,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31'
        ',32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,60/123'
    )

def format_request_tempDOsal(start, end):
    '''
    Creating URL string for requesting water quality data from the Chesapeake Bay 
    Program.
    
    Start and end date formatted as `month-day-year`, or '%m-%d-%Y' 
    using https://strftime.org/
    '''
    return (
        'https://datahub.chesapeakebay.net/api.CSV/WaterQuality/WaterQuality/'
        f'{start}/{end}/0/2,4,6/12,13,15,35,36,2,3,7,33,34,23,24/HUC8/'
        '2,4,6,7,8,9,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31'
        ',32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,60/31,123,83'
    )

def remove_blank_lines(response_text):
    """
    Remove blank lines from the response text.
    """
    lines = response_text.split("\r\n")
    non_empty_lines = [line for line in lines if line.strip() != ""]
    cleaned_text = "\n".join(non_empty_lines)
    return cleaned_text

format_request = format_request_tempDOsal
# The API seems to get overwhelmed when requesting the full 20 years of data at once.
# The request is instead split into one request per decade and the data frames are merged.

# Create temporary filepaths
scratch_dir = tempfile.TemporaryDirectory()
decade1_path = Path(scratch_dir.name, "decade1.csv")
decade2_path = Path(scratch_dir.name, "decade2.csv")
decade3_path = Path(scratch_dir.name, "decade3.csv")

# # Request #1 -- ~2002-2008
# response = requests.get(format_request(start_date.strftime("%m-%d-%Y"), "12-31-2008"))
# response.raise_for_status()  # ensure we notice bad responses
# with open(decade1_path, "w", encoding="utf-8") as f:
#     f.write(remove_blank_lines(response.text))

# # Request #2 -- ~2009-2015
# response = requests.get(format_request("01-01-2009", "12-31-2015"))
# response.raise_for_status()  # ensure we notice bad responses
# with open(decade2_path, "w", encoding="utf-8") as f:
#     f.write(remove_blank_lines(response.text))

# # Request #3 -- ~2016-2022
# response = requests.get(format_request("01-01-2016", end_date.strftime("%m-%d-%Y")))
# response.raise_for_status()  # ensure we notice bad responses
# with open(decade3_path, "w", encoding="utf-8") as f:
#     f.write(remove_blank_lines(response.text))

from dateutil.relativedelta import relativedelta

start_date = datetime.strptime("01-01-2002", "%m-%d-%Y")
end_date = datetime.strptime("12-31-2022", "%m-%d-%Y")
current_date = start_date

while current_date <= end_date:
    next_month_date = current_date + relativedelta(months=1)
    response = requests.get(
        format_request(
            current_date.strftime("%m-%d-%Y"), next_month_date.strftime("%m-%d-%Y")
        )
    )
    response.raise_for_status()  # ensure we notice bad responses

    # Save the data to a file named with the current month and year
    file_path = f"cpb_waterquality_{current_date.strftime('%Y_%m')}.csv"
    file_path = Path(scratch_dir.name, file_path)
    with Path.open(file_path, "w", encoding="utf-8") as f:
        f.write(remove_blank_lines(response.text))

    # Move to the next month
    current_date = next_month_date


In [ ]:
# Open the csvs using pandas
# decade1_df = pd.read_csv(decade1_path)
# decade2_df = pd.read_csv(decade2_path)
# decade3_df = pd.read_csv(decade2_path)

# Combine the datasets
# full_df = pd.concat([decade1_df, decade2_df, decade3_df])

# Combine all the csvs found in the scratch directory
full_df = pd.concat(
    [
        pd.read_csv(Path(scratch_dir.name, file))
        for file in os.listdir(scratch_dir.name)
    ]
)
# full_df = pd.concat(
#     [pd.read_csv(file) for file in os.listdir(scratch_dir.name) if file.endswith(".csv")]
# )

# Sort by date and reset the index
full_df.SampleDate = pd.to_datetime(full_df.SampleDate)
full_df = full_df.sort_values("SampleDate").reset_index(drop=True)

# Remove rows with null temperature values
full_df = full_df[~full_df.MeasureValue.isnull()]


In [5]:
# Save to the raw data folder
filename = ("WaterQuality_ChesapeakeBayProgram_{}_{}_TempDOSal.csv").format(
    start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d")
)

# check if the folder exists, if not create it
if not Path.exists(Path(REPO_ROOT, "data", "raw")):
    Path.mkdir(Path(REPO_ROOT, "data", "raw"))

output_path = Path(REPO_ROOT, "data", "raw", filename)

full_df.to_csv(output_path, index=False)


In [10]:
# Delete the temporary directory
scratch_dir.cleanup()
